In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from treemort.utils.config import setup

config_file_path = "../configs/flair_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/dead_trees/Finland/RGBNIR/25cm"
conf.output_dir = os.path.join("..", conf.output_dir)
conf.resume = False

print(conf)

2024-10-02 16:06:39,850 - INFO - Using config file: ../configs/flair_unet_bs8_cs256.txt
2024-10-02 16:06:39,855 - INFO - Configuration successfully loaded.


Namespace(model='flair_unet', backbone=None, model_weights='best', learning_rate=0.0001, activation='sigmoid', loss='hybrid', threshold=0.5, class_weights=[0.1, 0.9], epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.2, test_size=0.1, resume=False, data_folder='/Users/anisr/Documents/dead_trees/Finland/RGBNIR/25cm', hdf5_file='Finland_RGBNIR_25cm.h5', input_channels=4, output_channels=1, output_dir='../output/flair_unet')


In [3]:
from treemort.main import run

eval_only = False

run(conf, eval_only)

/Users/anisr/Documents/TreeSeg/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/anisr/Documents/TreeSeg/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-02 16:06:44,863 - INFO - Output directory already exists: ../output/flair_unet
2024-10-02 16:06:44,864 - INFO - Using device: cpu
2024-10-02 16:06:44,864 - INFO - Preparing datasets...
2024-10-02 16:06:45,219 - INFO - Datasets prepared: Train(80), Val(31), Test(15)
2024-10-02 16:06:45,220 - INFO - Loading or resuming model...
2024-10-02 16:06:45,220 - INFO - Building model...
2024-10-02 16:06:46,283 - INFO - flair_unet model configured.
202

KeyboardInterrupt: 

In [ ]:
import h5py

from treemort.data.loader import prepare_datasets
from treemort.utils.config import setup
from treemort.utils.plots import plot_examples

config_file_path = "../configs/hcfnet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/AerialImages"

hdf5_file_path = os.path.join(conf.data_folder, conf.hdf5_file)
with h5py.File(hdf5_file_path, 'r') as hf:
    keys = list(hf.keys())

train_dataset, val_dataset, test_dataset, _ = prepare_datasets(conf)

plot_examples(train_dataset, num_examples=5)